In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col, array_contains

spark = SparkSession.builder.appName('adq').getOrCreate()

In [2]:
delimeter = ","
df = spark.read.format("csv").option("inferSchema", True).option("header", True).option("sep", delimiter).load("TSLA.csv")

df.printSchema()
df.show()

NameError: name 'delimiter' is not defined